In [11]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from fastprogress.fastprogress import progress_bar
import pandas as pd

cwd = os.getcwd()
parentDir = os.path.dirname( cwd )
sys.path.append(parentDir)

In [18]:
# Load cylinder data
data_root = os.path.join(parentDir, 'data', 'raw', 'QSM', 'detailed')
noise_data_root = os.path.join(parentDir, 'data', 'noised', 'cloud')
QSMs = [os.path.join(data_root, path) for path in os.listdir(data_root) if path.endswith('.csv')]

for qsm in QSMs:
    # Extract filename from QSM path
    qsm_filename = os.path.basename(qsm)  # Example: "33_22_000000.csv"
    base_name = "_".join(qsm_filename.split("_")[:2])  # Extracts "33_22"
    # Define save paths
    npy_path = os.path.join(noise_data_root, f"{base_name}.npy")
    txt_path = os.path.join(noise_data_root, f"{base_name}.txt")

    cylinders = pd.read_csv(qsm)
    cylinders.columns = cylinders.columns.str.strip()

    # Extract cylinder parameters
    start = cylinders[['startX', 'startY', 'startZ']].values  # (N, 3)
    end = cylinders[['endX', 'endY', 'endZ']].values  # (N, 3)
    radius = cylinders['radius'].values  # (N,)
    axis = end - start
    axis_length = np.linalg.norm(axis, axis=1)  # (N,)
    axis_unit = axis / axis_length[:, None]  # Normalize per row

    # Compute number of points per cylinder
    density = 50  # Points per m²

    # Compute tree height
    tree_z_min = np.min(np.minimum(start[:, 2], end[:, 2]))
    tree_z_max = np.max(np.maximum(start[:, 2], end[:, 2]))
    tree_height = tree_z_max - tree_z_min

    # Compute relative height for each cylinder (0 = bottom, 1 = top)
    cylinder_height = (np.mean([start[:, 2], end[:, 2]], axis=0) - tree_z_min) / tree_height

    # Adjust density linearly (1 at bottom, 1/3 at top)
    density_factor = 1 - (3/4) * cylinder_height**0.33
    adjusted_density = density * density_factor  # Scale density per cylinder

    # Compute number of points per cylinder
    angles_per_cm = (2 * np.pi * radius * adjusted_density).astype(int)
    heights_per_cm = (axis_length * adjusted_density).astype(int)
    num_points = (angles_per_cm * heights_per_cm)

    # Create index array to map points to cylinders
    cylinder_ids = np.repeat(np.arange(len(cylinders)), num_points)

    # Generate random theta and height in a fully vectorized way
    theta = np.random.uniform(0, 2 * np.pi, size=cylinder_ids.shape)
    z = np.random.uniform(0, axis_length[cylinder_ids], size=cylinder_ids.shape)

    # Add random noise to the radius
    noise = np.random.lognormal(mean=-3, sigma=0.85, size=cylinder_ids.shape)  # Adjust parameters
    r_noisy = radius[cylinder_ids] + noise

    # Convert to local Cartesian coordinates
    x_local = r_noisy * np.cos(theta)
    y_local = r_noisy * np.sin(theta)
    z_local = z
    points_local = np.stack([x_local, y_local, z_local], axis=1)  # (Total_points, 3)

    # Compute rotation matrices for all cylinders
    z_axis = np.array([0, 0, 1])  # Local cylinder z-axis
    v = np.cross(z_axis, axis_unit)  # (N, 3)
    s = np.linalg.norm(v, axis=1)
    c = np.dot(z_axis, axis_unit.T)  # (N,)

    # Handle edge cases where v is zero (axis already aligned)
    v[s.flatten() == 0] = np.array([1, 0, 0])  # Set to arbitrary perpendicular vector

    # Compute skew-symmetric cross-product matrices Vx
    Vx = np.zeros((len(axis_unit), 3, 3))
    Vx[:, 0, 1] = -v[:, 2]
    Vx[:, 0, 2] = v[:, 1]
    Vx[:, 1, 0] = v[:, 2]
    Vx[:, 1, 2] = -v[:, 0]
    Vx[:, 2, 0] = -v[:, 1]
    Vx[:, 2, 1] = v[:, 0]

    # Compute rotation matrices using Rodrigues' formula
    I = np.eye(3)[None, :, :]  # Shape (1, 3, 3) to broadcast with Vx
    R = I + Vx + np.einsum('nij,njk->nik', Vx, Vx) * ((1 - c) / (s ** 2 + 1e-8))[:, None, None]

    # Rotate points
    points_rotated = np.einsum('nij,nj->ni', R[cylinder_ids], points_local)

    # Translate to world coordinates
    points_world = points_rotated + start[cylinder_ids]

    # Save as .npy (binary format for fast loading)
    np.save(npy_path, points_world)

    # # Save as .txt (for readability)
    if base_name == "32_2":
        np.savetxt(txt_path, points_world, fmt="%.6f", delimiter=" ")

In [9]:
import numpy as np
import plotly.graph_objects as go

def plot_point_cloud(npy_file, noise_threshold, output_html):
    # Load the point cloud
    points = np.load(npy_file)  # Shape: (N, 6)
    
    # Extract x, y, z coordinates and offset vectors
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_off, y_off, z_off = points[:, 3], points[:, 4], points[:, 5]
    
    # Compute the magnitude of the offset vectors
    offset_magnitude = np.sqrt(x_off**2 + y_off**2 + z_off**2)
    
    # Apply the threshold to color the points
    is_noise = offset_magnitude > noise_threshold
    colors = np.where(is_noise, 'red', 'grey')
    
    # Compute noise fractions
    noise_fraction = np.sum(is_noise) / len(points)
    non_noise_fraction = 1 - noise_fraction

    print(f"Noise: {noise_fraction:.3f}, non-noise: {non_noise_fraction:.3f}")
    
    # Create the 3D scatter plot
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=2, color=colors, opacity=0.8),
    ))
    
    # Set layout
    fig.update_layout(
        title="3D Point Cloud Visualization",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
        ),
    )
    
    # Save as HTML
    fig.write_html(output_html)
    print(f"Plot saved to {output_html}. Open this file in your browser.")

    return noise_fraction, non_noise_fraction

In [10]:
cloud_file = os.path.join(parentDir, 'data', 'labeled', 'noise', 'cloud', '32_2_labeled.npy')
html = os.path.join(parentDir, 'plots', 'DataAnalysis', 'art_noise.html')

plot_point_cloud(cloud_file, noise_threshold=0.05, output_html=html)

Plot saved to d:\Data_Science\Semester 4\Masterarbeit\Extracting-Tree-Morphology-From-Point-Clouds\plots\DataAnalysis\art_noise.html. Open this file in your browser.


(np.float64(0.4164741327686133), np.float64(0.5835258672313868))